In [ ]:
!pip install --upgrade google-meridian google-cloud-storage

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az
from google.cloud import storage

from meridian import constants
from meridian.data import load
from meridian.model import model, spec, prior_distribution
from meridian.analysis import optimizer, visualizer, summarizer

BUCKET_NAME = "tf-state-bucket-yuri-andrade"
CSV_FILENAME = "meridian_lite_weekly.csv"
GCS_CSV_PATH = f"gs://{BUCKET_NAME}/{CSV_FILENAME}"
GCS_PATH = f"gs://{BUCKET_NAME}/"

def upload_to_gcs(local_path, gcs_path):
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(local_path)

coord_to_columns = load.CoordToColumns(
    time='time',
    kpi='conversions',
    controls=['GQV'],
    media=['Display&DV360_impression', 'Google_Search_impression', 'Meta_Ads_impression',
           'Newspaper_impression', 'OOH_impression', 'Radio_impression', 'TikTok_Ads_impression',
           'TV_&_Offline_impression', 'YouTube_Ads_impression'],
    media_spend=['Display&DV360_spend', 'Google_Search_spend', 'Meta_Ads_spend',
                 'Newspaper_spend', 'OOH_spend', 'Radio_spend', 'TikTok_Ads_spend',
                 'TV_&_Offline_spend', 'YouTube_Ads_spend']
)

correct_media_to_channel = {media: media.split("_")[0] for media in coord_to_columns.media}
correct_media_spend_to_channel = {spend: spend.split("_")[0] for spend in coord_to_columns.media_spend}

loader = load.CsvDataLoader(
    csv_path=GCS_CSV_PATH,
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(0.2, 0.9, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)
mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

output_dir = "/content"
summary_html = os.path.join(output_dir, "lite_weekly_summary_output.html")
optimization_html = os.path.join(output_dir, "lite_weekly_optimization_output.html")
model_pkl = os.path.join(output_dir, "lite_weekly_saved_mmm.pkl")
media_csv = os.path.join(output_dir, "lite_weekly_media_summary.csv")

start_date = '2022-01-25'
end_date = '2024-12-31'

mmm_summarizer = summarizer.Summarizer(mmm)
mmm_summarizer.output_model_results_summary("lite_weekly_summary_output.html", output_dir, start_date, end_date)

media_summary = visualizer.MediaSummary(mmm)
summary_table = media_summary.summary_table()
df_summary = pd.DataFrame(summary_table)
df_summary.to_csv(media_csv, index=False)

budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize(use_kpi=True)
optimization_results.output_optimization_summary("lite_weekly_optimization_output.html", output_dir)

model.save_mmm(mmm, model_pkl)

upload_to_gcs(summary_html, "outputs/lite_weekly_summary_output.html")
upload_to_gcs(optimization_html, "outputs/lite_weekly_optimization_output.html")
upload_to_gcs(model_pkl, "outputs/lite_weekly_saved_mmm.pkl")
upload_to_gcs(media_csv, "lite_weekly_media_summary.csv")